In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [ ]:
# Create a SparkSession
spark = SparkSession.builder\
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.iceberg.type", "hadoop") \
        .config("spark.sql.catalog.iceberg.warehouse", WAREHOUSE_PATH) \
        .getOrCreate()

In [ ]:
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
# Set the MinIO access key, secret key, endpoint, and other configurations
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "admin")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

In [ ]:
# Read a JSON file from an MinIO bucket using the access key, secret key, 
# and endpoint configured above
df = spark.read.option("header", "true") \
    .csv(f"s3a://{minio_bucket}/vendas/vendas.csv")
# show data
df.show()

In [ ]:
# Write DataFrame to Iceberg
df.writeTo("demo.sales.vendas").create()

In [ ]:
spark.sql("SHOW DATABASES").show()

In [ ]:
spark.stop()